In [ ]:
import pandas as pd

# ==========================================
# 🎛️ TWEAKABLE PARAMETERS
# ==========================================
TARGET_TEMP = 25.0

# How aggressively should temperature pull the color toward Blue (0) or Red (100)?
HOT_PENALTY_MULT = 2.5  # Increase to push warm cities closer to 100 (Red)
COLD_PENALTY_MULT = 1.5 # Increase to push cool cities closer to 0 (Blue)

# How aggressively should rain/humidity shrink the bubble size?
RAIN_PENALTY_MULT = 5.0
HUMID_PENALTY_MULT = 0.3
# ==========================================

# 1. Load Data & Filter
df_weather = pd.read_csv("../data/raw/weather_data.csv")
df_planning = df_weather[df_weather['day_offset'] >= 2].copy()

# 2. Dynamic Functions
def test_climate_index(row):
    T = row['temp_day']
    if T > TARGET_TEMP:
        index = 50 + ((T - TARGET_TEMP) * HOT_PENALTY_MULT)
    else:
        index = 50 - ((TARGET_TEMP - T) * COLD_PENALTY_MULT)
    return max(0, min(100, index))

def test_weather_score(row):
    T = row['temp_day']
    if T > TARGET_TEMP:
        temp_penalty = (T - TARGET_TEMP) * 2 * HOT_PENALTY_MULT
    else:
        temp_penalty = (TARGET_TEMP - T) * 2 * COLD_PENALTY_MULT
        
    rain_penalty = row['rain'] * RAIN_PENALTY_MULT 
    humidity_penalty = max(0, (row['humidity'] - 60) * HUMID_PENALTY_MULT)
    
    return max(0, min(100, 100 - temp_penalty - rain_penalty - humidity_penalty))

# 3. Apply Calculations
df_planning['climate_index'] = df_planning.apply(test_climate_index, axis=1)
df_planning['weather_score'] = df_planning.apply(test_weather_score, axis=1)

# 4. Group by City to see the final map values
city_summary = df_planning.groupby('city').agg({
    'temp_day': 'mean',
    'rain': 'sum',
    'climate_index': 'mean',
    'weather_score': 'mean'
}).reset_index()

# Sort by Climate Index so we can see the spread from Coldest to Hottest
city_summary = city_summary.sort_values('climate_index')

# Display plain text formatting for easy reading
print("🌡️ CLIMATE INDEX SPREAD (0=Blue/Cold, 50=Green/Perfect, 100=Red/Hot)")
print("-" * 75)

# Round the numbers to 1 decimal place and display normally
display(city_summary.round(1))

🌡️ CLIMATE INDEX SPREAD (0=Blue/Cold, 50=Green/Perfect, 100=Red/Hot)
---------------------------------------------------------------------------


,city,temp_day,rain,climate_index,weather_score
13,Chateau du Haut Koenigsbourg,8.8,5.0,25.8,42.6
19,Grenoble,9.6,0.2,26.9,44.0
3,Annecy,10.4,0.7,28.1,50.5
21,Le Havre,10.9,6.8,28.8,44.0
17,Eguisheim,11.1,2.7,29.2,52.2
32,Strasbourg,11.4,6.9,29.6,50.0
4,Ariege,11.4,0.0,29.7,58.4
15,Colmar,11.5,3.0,29.8,53.2
18,Gorges du Verdon,11.6,0.0,29.9,59.8
8,Besancon,11.6,6.3,29.9,49.2


In [5]:
import pandas as pd
# 1. Load the raw weather data
df_weather = pd.read_csv("../data/processed/booking_data_enriched.csv")

df_weather.describe(include='all')

,city,hotel_name,url,score,description,hotel_lat,hotel_lon
count,700,700,700,687,700,700.000000,700.000000
unique,35,699,700,47,699,NaN,NaN
top,Mont Saint Michel,Hotel SPA Husseren Collections - Proche Colmar...,https://www.booking.com/hotel/fr/vert.html?aid...,8.58,"The hotel, with terraces southerly oriented, i...",NaN,NaN
freq,20,2,1,49,2,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,45.835590,3.402186
std,NaN,NaN,NaN,NaN,NaN,2.557156,2.903005
min,NaN,NaN,NaN,NaN,NaN,42.520945,-2.024303
25%,NaN,NaN,NaN,NaN,NaN,43.496283,1.354899
50%,NaN,NaN,NaN,NaN,NaN,45.189244,4.347114
75%,NaN,NaN,NaN,NaN,NaN,48.578388,5.716189
